In [ ]:
cost_table = [ [ 3, 2, 4, 6 ],
               [ 2, 3, 1, 2 ],
               [ 3, 2, 7, 4]
             ]

supply = [ 50, 40, 20 ]

demand = [ 30, 25, 30, 25 ]

In [ ]:
def calculate_cost(cost_table_, plan_):
  _cost = 0
  for _cell in plan_:
    _coords = _cell['Coordinates']
    _cost += cost_table_[_coords[0]][_coords[1]] * _cell['Value']
  return _cost

# Класс транспортной задачи

Добавим класс исключений, отражающий ошибки, возникающие при работе того или иного вычислительного метода:

In [ ]:
class MyClass:
  _var = 0

  def __init__(self, a_):
    self._a = a_

obj01 = MyClass(a_=5) # _var = 0
obj02 = MyClass(a_=7) # _var = 0

MyClass._var = 10       # _var = 10

print(obj01._var)
print(obj02._var)

10
10


In [ ]:
class MethodException(Exception):
  pass

In [ ]:
class TransportationProblem:
  # Классовая переменная, которая хранит номенклатуру успешных расчетных методов:
  _methods = set()

  def __init__(self, cost_table_, supply_, demand_):
    self._cost_table = cost_table_
    self._supply = supply_
    self._demand = demand_
    self._history = []

  def calculate(self, method):
    try:
      res_ = method(self._supply, self._demand) # Найти способ передавать методы с разным количеством аргументов
      try:
        self._history.append({'Method': method.__name__,
                            'Result': res_,
                            'Cost': calculate_cost(cost_table_=self._cost_table,
                                                   plan_=res_)})
      except AttributeError:
        self._history.append({'Method': 'Unknown',
                              'Result': res_,
                              'Cost': calculate_cost(cost_table_=self._cost_table,
                                                   plan_=res_)})
      TransportationProblem._methods.add(method)
    except Exception as e:
      TransportationProblem._methods.discard(method)
      if e is MethodException:
        print(f'Метод {method.__name__} не справился с расчетами')
      print(f'Метод {method.__name__} не может быть использован')
      print(e)
      pass

# Методы

## Метод северо-западного угла

Ниже приведена реализация метода:

In [ ]:
def north_west_corner(supply_, demand_):
  # Результирующий список, отвечающий плану:
  _res = []
  # Создадим копии коллекций, сохраняющих запасы и спросы:
  _supply = supply_.copy()
  _demand = demand_.copy()
  # Создаем итераторы по индексам контрагентов в таблице:
  i = 0
  j = 0
  # Основной цикл:
  while len(_res) < len(supply_) + len(demand_) - 1:
    # Находим минимум между текущими значениями спроса и предложения
    # для очередной пары контрагентов:
    v = min(_supply[i], _demand[j])
    # Зафиксируем в результатах:
    _res.append({'Coordinates': (i, j), 'Value': v})
    # Уменьшаем запас поставщика i и спрос потребителя j на величину v:
    _supply[i] -= v
    _demand[j] -= v
    # Определяем маршрут дальнейшего движения по таблице:
    if _supply[i] == 0 and i < len(_supply):
      i += 1
    elif _demand[j] == 0 and j < len(_demand):
      j += 1
    else:
      raise MethodException('Задача не имеет решения методом северо-западного угла')
  # Возвращаем список загруженных ячеек с указанием значений, которыми они загружены:
  return _res

In [ ]:
res = north_west_corner(supply_=supply, demand_=demand)
print(res)
print(calculate_cost(cost_table_=cost_table, plan_=res))

[{'Coordinates': (0, 0), 'Value': 30}, {'Coordinates': (0, 1), 'Value': 20}, {'Coordinates': (1, 1), 'Value': 5}, {'Coordinates': (1, 2), 'Value': 30}, {'Coordinates': (1, 3), 'Value': 5}, {'Coordinates': (2, 3), 'Value': 20}]
265


In [ ]:
print(north_west_corner)
print(north_west_corner.__name__)

<function north_west_corner at 0x7c101dd32560>
north_west_corner


## Метод минимального тарифа

In [ ]:
s = { 1, 1, 2, 3 }
print(s)

it = iter(s)
one = next(it)
print(one)

two = next(it)
print(two)

three = next(it)
print(three)

print(len(s))

{1, 2, 3}
1
2
3
3


In [ ]:
class MinCostMethodException(Exception):
  pass

In [ ]:
from itertools import product

In [ ]:
def find_min_cost(cost_table_, rows_excluded_, columns_excluded_):
  _rows_excluded = set(rows_excluded_)
  _columns_excluded = set(columns_excluded_)
  if len(_rows_excluded) >= len(cost_table_) or len(_columns_excluded) >= len(cost_table[0]):
    raise MinCostMethodException('Исключено слишком много строк или столбцов')
  _rows_set = set([i for i in range(len(cost_table_))]) - _rows_excluded
  _columns_set = set([j for j in range(len(cost_table_[0]))]) - _columns_excluded
  # Вход в цикл:
  _i_min = next(iter(_rows_set))
  _j_min = next(iter(_columns_set))
  _c = cost_table_[_i_min][_j_min]
  # Формируем набор всевозможных пар строкового и столбцового индексов,
  # которые остались после исключения:
  _coords = product(_rows_set, _columns_set)
  # Итерируем, чтобы найти минимальный тариф:
  for _coord in _coords:  # _coord = (i, j)
    _cost = cost_table_[_coord[0]][_coord[1]]
    if _cost < _c:
      _i_min = _coord[0]
      _j_min = _coord[1]
      _c = _cost
  return _i_min, _j_min, _c

In [ ]:
i, j, cost = find_min_cost(cost_table_=cost_table, rows_excluded_=[1], columns_excluded_=[])
print(f'Минимальная стоимость {cost} обнаружена в ячейке ({i}, {j})')

Минимальная стоимость 2 обнаружена в ячейке (0, 1)


In [ ]:
def min_cost_method(cost_table_, supply_, demand_):
  # Создаем копии входящих значений:
  _supply = supply_.copy()
  _demand = demand_.copy()
  # Переменные для формирования результата:
  _plan = []
  # Множества для хранения исключенных строк и столбцов:
  _rows_excluded = set()
  _columns_excluded = set()
  # Основной цикл, в котором условие остановки -- Условие 1 из презентации:
  while len(_plan) < len(supply_) + len(demand_) - 1:
    try:
      _i, _j, _min_cost = find_min_cost(cost_table_, _rows_excluded, _columns_excluded)
      # Обновление запаса и спроса:
      _s = _supply[_i]
      _d = _demand[_j]
      if _s <= _d:
        _rows_excluded.add(_i)
        _demand[_j] -= _s
      else:
        _columns_excluded.add(_j)
        _supply[_i] -= _d
       # Обновляем план перевозок:
      _plan.append({'Coordinates': (_i, _j), 'Value': min(_s, _d)})
    except MinCostMethodException:
      return _plan
  # Вовзращаем результат -- опорный план перевозок:
  return _plan

In [ ]:
pl_min = min_cost_method(cost_table_=cost_table, supply_=supply, demand_=demand)
print(pl_min)
print(calculate_cost(cost_table_=cost_table, plan_=pl_min))

[{'Coordinates': (1, 2), 'Value': 30}, {'Coordinates': (0, 1), 'Value': 25}, {'Coordinates': (1, 0), 'Value': 10}, {'Coordinates': (0, 0), 'Value': 20}, {'Coordinates': (2, 3), 'Value': 20}, {'Coordinates': (0, 3), 'Value': 5}]
270


## Метод Фогеля

In [ ]:
def penalty_in_list(input_list_):
  try:
    _l = input_list_.copy()
    _l.sort()
    return _l[1] - _l[0]
  except IndexError:
    return -1

In [ ]:
def analyze_table(cost_table_, rows_included_, columns_included_):
  # Вспомогательные функции:
  _get_col = lambda j, inds: [cost_table_[i][j] for i in inds]
  _get_row = lambda i, inds: [cost_table[i][j] for j in inds]
  # Функции для расчета штрафов на контрагентах:
  _penalty_in_col = lambda j: (j, max(penalty_in_list(_get_col(j, rows_included_)), -1))
  _penalty_in_row = lambda i: (i, max(penalty_in_list(_get_row(i, columns_included_)), -1))
  # Функции для расчета максимальных штрафов по таблице:
  _max_penalty_in_col = max([_penalty_in_col(j) for j in columns_included_], key=lambda el: el[1])
  _max_penalty_in_row = max([_penalty_in_row(i) for i in rows_included_], key=lambda el: el[1])
  # Анализ максимальных штрафов:
  if _max_penalty_in_row[0] >= _max_penalty_in_col[0]:
    _i = _max_penalty_in_row[0]
    _row = _get_row(_i, columns_included_)
    _min_cost = min(_row)
    return _i, [j for j, cost in enumerate(cost_table_[_i])
                                  if j in columns_included_ and cost == _min_cost][0], _min_cost # i, j, min_cost
  else:
    _j = _max_penalty_in_col[0]
    _col = _get_col(_j, rows_included_)
    _min_cost = min(_col)
    return [i for i, cost in enumerate(_get_col(_j, list(range(len(cost_table)))))
                                  if i in rows_included_ and cost == _min_cost][0], _j, _min_cost

In [ ]:
def vogel(cost_table_, supply_, demand_):
  _supply = supply_.copy()
  _demand = demand_.copy()
  print(_supply)
  _plan = []
  # Характеристики таблицы:
  _m = len(cost_table)
  _n = len(cost_table[0])
  # Разрешенные индексы:
  _row_inds = list(range(_m))
  _col_inds = list(range(_n))
  # Основной цикл:
  while len(_plan) < _m + _n - 1:
    # Ищем целевую ячейку:
    _i, _j, _cost = analyze_table(cost_table_, _row_inds, _col_inds)
    # print(f'Найденная ячейка: ({_i}, {_j}), стоимость: {_cost}')
    # Определяем груз, который отправляется между контрагентами:
    _cargo = min(_supply[_i], _demand[_j])
    # print(_cargo)
    # Осуществляем отправление груза:
    _supply[_i] -= _cargo
    _demand[_j] -= _cargo
    # print(f'Запас поставщика {_i}: {_supply[_i]}, спрос потребителя {_j}: {_demand[_j]}')
    # Фиксируем это в плане:
    _plan.append({'Coordinates': (_i, _j), 'Value': _cargo})
    # Вычеркиваем строку или столбец:
    if _supply[_i] == 0:
      _row_inds.remove(_i)
    elif _demand[_j] == 0:
      _col_inds.remove(_j)
    else:
      raise MethodException('Проблема!')
  # Результат:
  return _plan

In [ ]:
res01 = vogel(cost_table_=cost_table, supply_=supply, demand_=demand)
print(res01)
print(calculate_cost(cost_table_=cost_table, plan_=res01))

[{'Coordinates': (1, 2), 'Value': 30}, {'Coordinates': (1, 3), 'Value': 10}, {'Coordinates': (2, 3), 'Value': 15}, {'Coordinates': (0, 1), 'Value': 25}, {'Coordinates': (0, 0), 'Value': 25}, {'Coordinates': (2, 0), 'Value': 5}]
250


# Эксперименты

In [ ]:
from functools import partial

In [ ]:
tr_pr = TransportationProblem(cost_table_=cost_table,
                              supply_=supply,
                              demand_=demand)

tr_pr.calculate(method=partial(vogel, tr_pr._cost_table))

print(tr_pr._history)

[50, 40, 20]
[{'Method': 'Unknown', 'Result': [{'Coordinates': (1, 2), 'Value': 30}, {'Coordinates': (1, 3), 'Value': 10}, {'Coordinates': (2, 3), 'Value': 15}, {'Coordinates': (0, 1), 'Value': 25}, {'Coordinates': (0, 0), 'Value': 25}, {'Coordinates': (2, 0), 'Value': 5}], 'Cost': 250}]


In [ ]:
tr_pr.calculate(method=print)

[50, 40, 20] [30, 25, 30, 25]
Метод print не может быть использован
'NoneType' object is not iterable


In [ ]:
print([1,2,3], [4,5,6])

[1, 2, 3] [4, 5, 6]
